In [374]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout 
from keras.optimizers import SGD
#SGD Stochastic Gradient Descend
import random
import pandas as pd

In [375]:
lemmatizer = WordNetLemmatizer()

In [376]:
with open('intent.json','r') as f:
    data = json.load(f)
df = pd.DataFrame(data['intents'])

In [377]:
dict = []
documents = []
tags = []
df

,tag,patterns,responses
0,greeting,"[Hi, Hello, Hey there, Good morning, How are y...","[Hello! How can I help you today?, Hi there! W..."
1,weather,"[What's the weather like today?, Is it going t...",[The weather today is sunny with a high of 25°...
2,restaurant_recommendations,"[Can you recommend a good restaurant?, I'm loo...",[Sure! How about [Restaurant A]? They have gre...
3,product_information,"[Tell me about your products., What types of p...",[We offer a wide range of products including e...
4,booking_flights,"[I want to book a flight., Can you help me fin...",[Sure! Where would you like to fly to and from...
5,tech_support,"[I'm having trouble with my [device]., How do ...",[I'm sorry to hear that. What seems to be the ...
6,order_tracking,"[Where is my order?, Can you track my package?...",[Please provide your order number so I can che...
7,account_support,"[I forgot my password., How do I change my ema...",[You can reset your password through the passw...
8,appointment_booking,"[How do I schedule an appointment?, Can I book...",[You can schedule an appointment online throug...
9,general_info,"[What can you tell me about your company?, Whe...","[We are a company specializing in [industry].,..."


In [378]:
tags = df['tag'].unique()
questions = df['patterns']

In [379]:
questions

0     [Hi, Hello, Hey there, Good morning, How are y...
1     [What's the weather like today?, Is it going t...
2     [Can you recommend a good restaurant?, I'm loo...
3     [Tell me about your products., What types of p...
4     [I want to book a flight., Can you help me fin...
5     [I'm having trouble with my [device]., How do ...
6     [Where is my order?, Can you track my package?...
7     [I forgot my password., How do I change my ema...
8     [How do I schedule an appointment?, Can I book...
9     [What can you tell me about your company?, Whe...
10    [Are you hiring?, How can I apply for a job?, ...
11    [How can I provide feedback?, Do you welcome c...
12    [Are there any upcoming events?, What events a...
13    [What payment methods do you accept?, Can I pa...
14    [How long does shipping take?, Do you offer in...
15    [How can I contact customer support?, I need h...
16    [Do you have a loyalty program?, How can I joi...
17    [Do you provide insurance services?, What 

In [380]:
document = []
for index, row in df.iterrows():
    patterns = row['patterns']
    tag = row['tag']

    # Iterate over questions in each pattern
    for question in patterns:
        
        # Append [question, tag] to documents
        document.append([question, tag])
document

[['Hi', 'greeting'],
 ['Hello', 'greeting'],
 ['Hey there', 'greeting'],
 ['Good morning', 'greeting'],
 ['How are you?', 'greeting'],
 ["What's the weather like today?", 'weather'],
 ['Is it going to rain tomorrow?', 'weather'],
 ["How's the weather in [city]?", 'weather'],
 ["Tell me about today's weather forecast.", 'weather'],
 ['Will it be sunny this weekend?', 'weather'],
 ['Can you recommend a good restaurant?', 'restaurant_recommendations'],
 ["I'm looking for Italian food.", 'restaurant_recommendations'],
 ['Any sushi restaurants nearby?', 'restaurant_recommendations'],
 ['Where can I find a vegetarian restaurant?', 'restaurant_recommendations'],
 ["What's a popular place for brunch?", 'restaurant_recommendations'],
 ['Tell me about your products.', 'product_information'],
 ['What types of products do you offer?', 'product_information'],
 ['Can you help me choose a product?', 'product_information'],
 ['I need information on [product name].', 'product_information'],
 ['How much

In [381]:
documents = pd.DataFrame(document)
X = list(documents[0])
y = list(documents[1])
y

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'weather',
 'weather',
 'weather',
 'weather',
 'weather',
 'restaurant_recommendations',
 'restaurant_recommendations',
 'restaurant_recommendations',
 'restaurant_recommendations',
 'restaurant_recommendations',
 'product_information',
 'product_information',
 'product_information',
 'product_information',
 'product_information',
 'booking_flights',
 'booking_flights',
 'booking_flights',
 'booking_flights',
 'booking_flights',
 'tech_support',
 'tech_support',
 'tech_support',
 'tech_support',
 'tech_support',
 'order_tracking',
 'order_tracking',
 'order_tracking',
 'order_tracking',
 'order_tracking',
 'account_support',
 'account_support',
 'account_support',
 'account_support',
 'account_support',
 'appointment_booking',
 'appointment_booking',
 'appointment_booking',
 'appointment_booking',
 'appointment_booking',
 'general_info',
 'general_info',
 'general_info',
 'general_info',
 'general_info',
 'job_applicati

In [382]:
from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()

In [383]:
data = {
    'question': X,
    'tag':y
}

df = pd.DataFrame(data)

In [384]:
df

,question,tag
0,Hi,greeting
1,Hello,greeting
2,Hey there,greeting
3,Good morning,greeting
4,How are you?,greeting
...,...,...
95,Do you offer educational courses?,educational_courses
96,What subjects do you teach?,educational_courses
97,How can I enroll in a course?,educational_courses
98,Are there online courses available?,educational_courses


In [385]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
vectorizer = TfidfVectorizer()

question_vectors = vectorizer.fit_transform(df['question']).toarray()
X = question_vectors
# One-hot encode the tag column
onehot_encoder = OneHotEncoder(sparse_output=False)
tag_onehot = onehot_encoder.fit_transform(df[['tag']])
y = tag_onehot
# Combine the TF-IDF vectors and one-hot encoded tags
combined_vectors = np.hstack((question_vectors, tag_onehot))

# Feature names for TF-IDF vectors
tfidf_feature_names = vectorizer.get_feature_names_out()

# Categories for one-hot encoded tags
onehot_categories = onehot_encoder.categories_[0]

In [397]:
# Print the results
print("Combined Vectors:\n", combined_vectors)
print("TF-IDF Feature Names:\n", tfidf_feature_names)
print("One-Hot Encoded Categories:\n", onehot_categories)
with open('word.pkl', 'wb') as pkl_file:
    pickle.dump(tfidf_feature_names, pkl_file)

# Save one-hot encoded categories to 'class.pkl'
with open('class.pkl', 'wb') as pkl_file:
    pickle.dump(onehot_categories, pkl_file)
len(combined_vectors[0])
combined_vectors[0]

Combined Vectors:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
TF-IDF Feature Names:
 ['about' 'accept' 'account' 'additional' 'address' 'advice' 'after' 'an'
 'any' 'apply' 'appointment' 'are' 'arrive' 'attend' 'available' 'be'
 'been' 'benefits' 'book' 'booking' 'brunch' 'business' 'can' 'card'
 'cash' 'certain' 'certificate' 'change' 'charges' 'choose' 'city'
 'claims' 'company' 'complaint' 'completing' 'connect' 'consultation'
 'contact' 'cost' 'course' 'courses' 'coverage' 'credit' 'customer' 'date'
 'deductible' 'delete' 'delivery' 'destination' 'device' 'do' 'doctor'
 'does' 'earn' 'educational' 'email' 'enroll' 'event' 'events' 'fee'
 'feedback' 'fees' 'find' 'fix' 'flight' 'flights' 'food' 'for' 'forecast'
 'forgot' 'free' 'get' 'going' 'good' 'happening' 'have' 'haven' 'having'
 'hello' 'help' 'hey' 'hi' 'hiring' 'host' 'hours' 'how' 'improvement'
 'in' 'information

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0.

In [387]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [388]:
from sklearn.model_selection import train_test_split

In [389]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33,random_state = 42)

In [390]:
# model = Sequential()
# model.add(Dense(128, input_dim=question_vectors.shape[1], activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(tag_onehot.shape[1], activation='softmax'))

# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model
# model.fit(question_vectors, tag_onehot, epochs=50, batch_size=8, verbose=2)

# # Evaluate the model
# loss, accuracy = model.evaluate(question_vectors, tag_onehot)
# print(f'Loss: {loss}, Accuracy: {accuracy}')

In [391]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

In [392]:
model = Sequential()
model.add(Dense(128, input_dim=question_vectors.shape[1], activation='relu',kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
# model.add(Dropout(0.1))
model.add(Dense(64, activation='relu',kernel_regularizer=l2(0.001)))
# model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(tag_onehot.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
hist = model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(question_vectors, tag_onehot)
print(f'Loss: {loss}, Accuracy: {accuracy}')
# argmax is used to conver one hot encoding back to original values
# the neural network model gives probabilities and teh one with maximum probability gets selected hence argmax is used
y_pred = model.predict(X_test).argmax(axis = 1)




Epoch 1/200


C:\Users\sarva\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 - 5s - 730ms/step - accuracy: 0.0299 - loss: 3.6366
Epoch 2/200
7/7 - 0s - 10ms/step - accuracy: 0.1194 - loss: 3.0804
Epoch 3/200
7/7 - 0s - 10ms/step - accuracy: 0.2537 - loss: 2.7015
Epoch 4/200
7/7 - 0s - 11ms/step - accuracy: 0.4627 - loss: 2.2406
Epoch 5/200
7/7 - 0s - 9ms/step - accuracy: 0.6716 - loss: 1.9779
Epoch 6/200
7/7 - 0s - 10ms/step - accuracy: 0.7910 - loss: 1.6452
Epoch 7/200
7/7 - 0s - 10ms/step - accuracy: 0.8955 - loss: 1.4353
Epoch 8/200
7/7 - 0s - 11ms/step - accuracy: 0.9104 - loss: 1.2235
Epoch 9/200
7/7 - 0s - 11ms/step - accuracy: 0.9701 - loss: 0.9797
Epoch 10/200
7/7 - 0s - 11ms/step - accuracy: 0.9403 - loss: 0.8897
Epoch 11/200
7/7 - 0s - 11ms/step - accuracy: 0.9851 - loss: 0.8164
Epoch 12/200
7/7 - 0s - 9ms/step - accuracy: 1.0000 - loss: 0.7061
Epoch 13/200
7/7 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.5777
Epoch 14/200
7/7 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.5435
Epoch 15/200
7/7 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.4374
Ep

In [393]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,precision_score, recall_score, f1_score

In [394]:
# the neural network model gives probabilities and teh one with maximum probability gets selected hence argmax is used
print(y_pred)
y_test_labels = y_test.argmax(axis=1)
print(y_test_labels)

[ 3 11 17 10 15  0 10 12  8 18 15 13 17 13 11 10 18 18  7  6  6  9 10  1
 14 10  1 10 19  3 10  7  9]
[12 10 17  7  1  0  2 12 16  8 15 13 17 13 11  8  3  3 16 13  6  9 18  1
 14 15  1  4 19 17 16  7  9]


In [395]:
from tensorflow.keras.models import save_model
accuracy_score(y_test_labels,y_pred)


0.48484848484848486

In [396]:
# save_model(model, 'chatbot_model.keras')